In [1]:
import sys
sys.path.append("../")

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import pandas as pd
import os,sys
from pathlib import Path
import json 
import logging
logging.basicConfig(format='%(asctime)s %(levelname)-7s %(message)s',
                    stream=sys.stderr, level=logging.INFO)
mpl_logger = logging.getLogger('matplotlib')
mpl_logger.setLevel(logging.INFO)

#General ML 
from sklearn.metrics import f1_score, confusion_matrix, accuracy_score, silhouette_score, classification_report
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import cross_val_score, cross_validate, StratifiedKFold, train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest, chi2
from itertools import combinations

#In-house Module Imports
from config import Configuration 
from datasets import EclipseSampledDataset, VoltaSampledDataset
from utils import *

In [4]:
user = "aksar"
logging.warning(f'Are you sure that you are: {user}?')

2022-02-18 17:50:24,017 WARNING Are you sure that you are: aksar?


In [5]:
#Update these if you are not the desired user
OUTPUT_DIR = f'/projectnb/peaclab-mon/{user}/unseen_experiments'
SYSTEM = 'volta'
EXP_NAME = 'tpds_experiments'
CV_INDEX = 0
FEATURE_SELECTION = False
SCALER = 'None' #For now, do the scaling inside the notebook, then you can move that to the class function
MODEL_CONFIG = 'experiment_3' #rf_tuncer or rf_tuncer_worst_case
logging.warning('Results will be generated in {}, double check please!'.format(MODEL_CONFIG))

2022-02-18 17:50:24,048 WARNING Results will be generated in experiment_3, double check please!


In [6]:
conf = Configuration(ipython=True,
                     overrides={
                         'output_dir': Path(OUTPUT_DIR), #change
                         'system' : SYSTEM,
                         'exp_name':EXP_NAME,                                                  
                         'cv_fold':CV_INDEX, 
                         'model_config': MODEL_CONFIG,
                     })

with open(str(conf['experiment_dir']) + '/anom_dict.json') as f:
    ANOM_DICT = json.load(f)
with open(str(conf['experiment_dir']) + '/app_dict.json') as f:
    APP_DICT = json.load(f)    
    
APP_REVERSE_DICT = {}
for app_name, app_encoding in APP_DICT.items():
    APP_REVERSE_DICT[app_encoding] = app_name    

ANOM_REVERSE_DICT = {}
for anom_name, anom_encoding in ANOM_DICT.items():
    ANOM_REVERSE_DICT[anom_encoding] = anom_name        

2022-02-18 17:50:24,080 WARNING You set windowing False, but you are trying to define window_size parameter, it is automatically set to 0. Please double check.
2022-02-18 17:50:24,080 INFO    Setting directory names
2022-02-18 17:50:24,092 INFO    Saving configuration as CSV


# The configuration used for this run:
# {'cv_fold': 0,
#  'exp_name': 'tpds_experiments',
#  'experiment_dir': PosixPath('/projectnb/peaclab-mon/aksar/unseen_experiments/volta/tpds_experiments'),
#  'feature_extract': False,
#  'feature_select': False,
#  'hdf_data_path': PosixPath('/projectnb/peaclab-mon/aksar/datasets/tpds_data_hdfs'),
#  'metadata_path': None,
#  'model_config': 'experiment_3',
#  'model_config_dir': PosixPath('/projectnb/peaclab-mon/aksar/unseen_experiments/volta/tpds_experiments/CV_0/experiment_3'),
#  'model_dir': PosixPath('/projectnb/peaclab-mon/aksar/unseen_experiments/volta/tpds_experiments/CV_0/experiment_3/model'),
#  'num_split': 5,
#  'operation': 'read',
#  'output_dir': PosixPath('/projectnb/peaclab-mon/aksar/unseen_experiments/volta'),
#  'plots_dir': PosixPath('/projectnb/peaclab-mon/aksar/unseen_experiments/volta/tpds_experiments/CV_0/experiment_3/model/plots'),
#  'processed_ldms_data_path': None,
#  'raw_ldms_data_path': None,
#  'results_dir': Po

In [7]:
if SYSTEM == 'eclipse':
    
    eclipseDataset = EclipseSampledDataset(conf)
    train_data, train_label, test_data, test_label = eclipseDataset.load_dataset(scaler=SCALER,borghesi=False,tsfresh = True)

elif SYSTEM == 'volta':
    voltaDataset = VoltaSampledDataset(conf)
    train_data, train_label, test_data, test_label = voltaDataset.load_dataset(scaler=SCALER,borghesi=False)
    
assert list(train_data.index) == list(train_label.index) #check the order of the labels     
assert list(test_data.index) == list(test_label.index) #check the order of the labels    

if FEATURE_SELECTION:
    selected_features = pd.read_csv(conf['experiment_dir'] / 'selected_features.csv')
    train_data = train_data[list(selected_features['0'].values)]
    test_data = test_data[list(selected_features['0'].values)]
    
train_label['anom_names'] = train_label.apply(lambda x: ANOM_REVERSE_DICT[x['anom']], axis=1)
train_label['app_names']=train_label['app'].apply(lambda x: APP_REVERSE_DICT[x])
test_label['anom_names'] = test_label.apply(lambda x: ANOM_REVERSE_DICT[x['anom']], axis=1)
test_label['app_names']=test_label['app'].apply(lambda x: APP_REVERSE_DICT[x])

all_data = pd.concat([train_data, test_data])
all_data = all_data.dropna(axis=1, how='any')
all_label = pd.concat([train_label,test_label])

train_data = all_data.loc[train_label.index]
test_data = all_data.loc[test_label.index]
    
logging.info("Train data shape %s",train_data.shape)
logging.info("Train label shape %s",train_label.shape)
logging.info("Test data shape %s",test_data.shape)  
logging.info("Test label shape %s",test_label.shape)

logging.info("Train data label dist: \n%s",train_label['anom'].value_counts())
logging.info("Test data label dist: \n%s",test_label['anom'].value_counts())        

2022-02-18 17:50:24,138 INFO    BaseDataset Class Initialization
2022-02-18 17:50:24,138 INFO    HPCDataset Class Initialization
2022-02-18 17:50:24,139 INFO    VoltaSampledDataset Class Initialization
2022-02-18 17:50:24,656 INFO    Note: NumExpr detected 16 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
2022-02-18 17:50:26,555 INFO    Train data shape (14640, 1441)
2022-02-18 17:50:26,556 INFO    Train label shape (14640, 2)
2022-02-18 17:50:26,557 INFO    Test data shape (6275, 1441)
2022-02-18 17:50:26,558 INFO    Test label shape (6275, 2)
2022-02-18 17:50:26,559 WARNING Beware that no scaling method is applied
2022-02-18 17:50:27,803 INFO    Train data shape (14640, 1441)
2022-02-18 17:50:27,806 INFO    Train label shape (14640, 4)
2022-02-18 17:50:27,806 INFO    Test data shape (6275, 1441)
2022-02-18 17:50:27,807 INFO    Test label shape (6275, 4)
2022-02-18 17:50:27,817 INFO    Train data label dist: 
0    13281
2      342
1      342
3      338
4      3

## Unknown App Experiments

In [21]:
SCALER = 'MinMax'

In [9]:
nas_apps = ['lu','sp','ft','bt','cg','mg']
mantevo_apps = ['miniMD','CoMD','miniGhost','miniAMR']
other_apps = ['kripke']

all_app_names = list(APP_DICT.keys())

assert set(nas_apps + mantevo_apps + other_apps) == set(all_app_names)


cluster_one = ['ft','cg','mg'] #126
cluster_two = ['miniAMR','lu','miniGhost'] #126
cluster_three = ['sp','bt','miniMD','kripke','CoMD'] #max 6 

In [10]:
all_test_app_groups = []
all_test_app_sets = []
for cluster in [cluster_one,cluster_two,cluster_three]:    
    all_test_app_groups.append(cluster)
    all_test_app_sets.append(set(cluster))

In [17]:
#This doesn't hold the unique combinations, however it should have the total number
train_app_combinations = []

#90 different test_apps combination
for test_apps in all_test_app_groups:
    
    train_apps_all = list(set(all_app_names) - set(test_apps))
    #logging.info("Test apps: %s", test_apps)
    #logging.info("All Train apps: %s", train_apps_all)
    
    num_train_unknowns = [2,4,6]

    #126 or 31 different train_apps combination
    for num_train_unknown in num_train_unknowns:
        for train_apps in combinations(train_apps_all, num_train_unknown):
            train_app_combinations.append(train_apps)

In [18]:
"""
This code blocks read the processed filenames so far and creates a dict using (train_apps,test_apps) as a key
It can also detect cases which are processed multiple times with different names
"""

not_covered_combinations = []
unique_counter_dict = {}
extra_filenames = []
delete_extra_files = False

for file in listdir(conf['results_dir']):
    
    if "result_dict" in file:

        filename = file.split('.')[0]
        splitted_filename = file.split('#')    

        train_apps = set(splitted_filename[0].split(':')[1].split("-"))
        test_apps = set(splitted_filename[1].split(':')[1].split("-"))
        
        if not (frozenset(train_apps),frozenset(test_apps)) in unique_counter_dict:
            unique_counter_dict[(frozenset(train_apps),frozenset(test_apps))] = 1
        else:
            unique_counter_dict[(frozenset(train_apps),frozenset(test_apps))] += 1
            extra_filenames.append(file)
                    
        if (not test_apps in all_test_app_sets) and (not train_apps in train_app_combinations):
            not_covered_combinations.append([train_apps,test_apps])
                        
sorted_unique_counter_dict = dict(sorted(unique_counter_dict.items(), key=lambda x: x[1], reverse=True))            

if delete_extra_files:
    
    for filename in extra_filenames:

        generic_filename = "#".join(filename.split("#")[:-1])

        os.remove(Path(conf['results_dir']) / filename)
        os.remove(Path(conf['results_dir']) / (generic_filename + "#alert_dict.json"))    

In [22]:
counter = 0
for test_apps in all_test_app_groups:
    
    train_apps_all = list(set(all_app_names) - set(test_apps))
    logging.info("Test apps: %s", test_apps)
    logging.info("All Train apps: %s", train_apps_all)
    
    test_apps_label = all_label[all_label['app_names'].isin(test_apps)]
    assert set(test_apps_label['app_names'].unique()) == set(test_apps)
    
    test_apps_data = all_data.loc[test_apps_label.index]
    assert list(test_apps_data.index) == list(test_apps_label.index) #check the order of the labels       
    
    num_train_unknowns = [2,4,6]
                
    for num_train_unknown in num_train_unknowns:    
        
        for train_apps_selected in combinations(train_apps_all, num_train_unknown):        
            
            train_apps_selected = list(train_apps_selected)
            logging.info("Selected train apps: %s", train_apps_selected)
                        
            if not (frozenset(train_apps_selected),frozenset(test_apps)) in unique_counter_dict:                
            
                train_apps_label = all_label[all_label['app_names'].isin(train_apps_selected)]
                train_apps_data = all_data.loc[train_apps_label.index]
                assert list(train_apps_data.index) == list(train_apps_label.index) #check the order of the labels 

                if SCALER == 'MinMax':

                    scaler = MinMaxScaler()    

                elif SCALER == 'Standard':

                    scaler = StandardScaler()
                    
                clf = RandomForestClassifier(random_state=0)
                
                pipeline = Pipeline([('transformer', scaler), ('estimator', clf)])  
                
                fit_clf = pipeline.fit(train_apps_data, train_apps_label['anom'])

                pred_test = pipeline.predict(test_apps_data)                
                logging.info("\n %s",classification_report(test_apps_label['anom'],pred_test, target_names=ANOM_DICT))
                
                result_filename = "train:" + "-".join(train_apps_selected) + "#test:" + "-".join(test_apps) \
                            + "#model:" + clf.__class__.__name__ + "#cv:" + str(CV_INDEX)                    
                logging.info(result_filename)
                result_dict = classification_report(test_apps_label['anom'],pred_test, target_names=ANOM_DICT,output_dict=True)

                with open(Path(conf['results_dir']) / (result_filename + "#result_dict.json"), "w") as fp:
                    json.dump(result_dict, fp)

                FAR_AMR_Calculate(true_label=test_apps_label['anom'],pred_label=pred_test,
                                  result_dir=conf['results_dir'],
                                  save_name=result_filename,save=True)                                   
            else: 
                counter += 1
                logging.info("File exists")

2022-02-18 18:04:24,909 INFO    Test apps: ['ft', 'cg', 'mg']
2022-02-18 18:04:24,910 INFO    All Train apps: ['sp', 'CoMD', 'miniAMR', 'bt', 'kripke', 'miniMD', 'miniGhost', 'lu']
2022-02-18 18:04:24,961 INFO    Selected train apps: ['sp', 'CoMD']
2022-02-18 18:04:31,387 INFO    
               precision    recall  f1-score   support

        none       0.97      0.67      0.79      5015
       dcopy       0.98      0.99      0.99       130
    memeater       1.00      1.00      1.00       125
        leak       1.00      1.00      1.00       125
        dial       0.02      0.32      0.04       125

    accuracy                           0.68      5520
   macro avg       0.80      0.80      0.76      5520
weighted avg       0.95      0.68      0.79      5520

2022-02-18 18:04:31,388 INFO    train:sp-CoMD#test:ft-cg-mg#model:RandomForestClassifier#cv:0
2022-02-18 18:04:31,408 INFO    Total misclassified normal runs: 1675, Total normal runs 5015 
2022-02-18 18:04:31,408 INFO    FAR: 0.

2022-02-18 18:05:14,140 INFO    train:CoMD-miniAMR#test:ft-cg-mg#model:RandomForestClassifier#cv:0
2022-02-18 18:05:14,153 INFO    Total misclassified normal runs: 1626, Total normal runs 5015 
2022-02-18 18:05:14,154 INFO    FAR: 0.3242273180458624
2022-02-18 18:05:14,155 INFO    Total misclassified anom runs: 55, Total anom runs 505 
2022-02-18 18:05:14,155 INFO    AMR: 0.10891089108910891
2022-02-18 18:05:14,157 INFO    Selected train apps: ['CoMD', 'bt']
2022-02-18 18:05:20,204 INFO    
               precision    recall  f1-score   support

        none       0.98      0.74      0.84      5015
       dcopy       0.96      0.99      0.97       130
    memeater       0.32      1.00      0.48       125
        leak       1.00      1.00      1.00       125
        dial       0.03      0.22      0.05       125

    accuracy                           0.74      5520
   macro avg       0.65      0.79      0.67      5520
weighted avg       0.94      0.74      0.82      5520

2022-02-18 18:

2022-02-18 18:05:53,867 INFO    train:miniAMR-miniMD#test:ft-cg-mg#model:RandomForestClassifier#cv:0
2022-02-18 18:05:53,880 INFO    Total misclassified normal runs: 2791, Total normal runs 5015 
2022-02-18 18:05:53,881 INFO    FAR: 0.556530408773679
2022-02-18 18:05:53,882 INFO    Total misclassified anom runs: 65, Total anom runs 505 
2022-02-18 18:05:53,882 INFO    AMR: 0.12871287128712872
2022-02-18 18:05:53,884 INFO    Selected train apps: ['miniAMR', 'miniGhost']
2022-02-18 18:05:57,961 INFO    
               precision    recall  f1-score   support

        none       0.98      1.00      0.99      5015
       dcopy       1.00      0.98      0.99       130
    memeater       1.00      1.00      1.00       125
        leak       1.00      1.00      1.00       125
        dial       0.17      0.01      0.02       125

    accuracy                           0.98      5520
   macro avg       0.83      0.80      0.80      5520
weighted avg       0.96      0.98      0.97      5520

202

2022-02-18 18:06:38,972 INFO    train:kripke-miniGhost#test:ft-cg-mg#model:RandomForestClassifier#cv:0
2022-02-18 18:06:38,985 INFO    Total misclassified normal runs: 3883, Total normal runs 5015 
2022-02-18 18:06:38,985 INFO    FAR: 0.7742771684945164
2022-02-18 18:06:38,986 INFO    Total misclassified anom runs: 29, Total anom runs 505 
2022-02-18 18:06:38,987 INFO    AMR: 0.05742574257425743
2022-02-18 18:06:38,989 INFO    Selected train apps: ['kripke', 'lu']
2022-02-18 18:06:44,047 INFO    
               precision    recall  f1-score   support

        none       0.98      1.00      0.99      5015
       dcopy       1.00      0.93      0.96       130
    memeater       0.94      1.00      0.97       125
        leak       1.00      1.00      1.00       125
        dial       0.00      0.00      0.00       125

    accuracy                           0.98      5520
   macro avg       0.78      0.79      0.78      5520
weighted avg       0.95      0.98      0.96      5520

2022-02-

2022-02-18 18:07:35,630 INFO    train:sp-CoMD-miniAMR-miniMD#test:ft-cg-mg#model:RandomForestClassifier#cv:0
2022-02-18 18:07:35,645 INFO    Total misclassified normal runs: 0, Total normal runs 5015 
2022-02-18 18:07:35,645 INFO    FAR: 0.0
2022-02-18 18:07:35,646 INFO    Total misclassified anom runs: 128, Total anom runs 505 
2022-02-18 18:07:35,647 INFO    AMR: 0.25346534653465347
2022-02-18 18:07:35,648 INFO    Selected train apps: ['sp', 'CoMD', 'miniAMR', 'miniGhost']
/project/peaclab-mon/monitoring_venv/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
2022-02-18 18:07:47,425 INFO    
               precision    recall  f1-score   support

        none       0.98      1.00      0.99      5015
       dcopy       1.00      0.98      0

...

2022-02-18 18:59:31,220 INFO    AMR: 0.06666666666666667
2022-02-18 18:59:31,222 INFO    Selected train apps: ['ft', 'CoMD', 'mg', 'bt', 'kripke', 'miniMD']
2022-02-18 18:59:56,169 INFO    
               precision    recall  f1-score   support

        none       1.00      1.00      1.00      5265
       dcopy       1.00      0.96      0.98       135
    memeater       1.00      1.00      1.00       135
        leak       1.00      1.00      1.00       135
        dial       0.95      0.91      0.93       135

    accuracy                           1.00      5805
   macro avg       0.99      0.97      0.98      5805
weighted avg       1.00      1.00      1.00      5805

2022-02-18 18:59:56,170 INFO    train:ft-CoMD-mg-bt-kripke-miniMD#test:miniAMR-lu-miniGhost#model:RandomForestClassifier#cv:0
2022-02-18 18:59:56,186 INFO    Total misclassified normal runs: 0, Total normal runs 5265 
2022-02-18 18:59:56,187 INFO    FAR: 0.0
2022-02-18 18:59:56,188 INFO    Total misclassified anom runs

2022-02-18 19:02:51,308 INFO    AMR: 0.016666666666666666
2022-02-18 19:02:51,310 INFO    Test apps: ['sp', 'bt', 'miniMD', 'kripke', 'CoMD']
2022-02-18 19:02:51,310 INFO    All Train apps: ['ft', 'cg', 'miniAMR', 'mg', 'miniGhost', 'lu']
2022-02-18 19:02:51,417 INFO    Selected train apps: ['ft', 'cg']
/project/peaclab-mon/monitoring_venv/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
2022-02-18 19:02:55,864 INFO    
               precision    recall  f1-score   support

        none       0.00      0.00      0.00      8700
       dcopy       0.33      1.00      0.49       225
    memeater       1.00      0.81      0.90       225
        leak       1.00      1.00      1.00       220
        dial       0.03      0.99      0.05       220

2022-02-18 19:03:38,355 INFO    
               precision    recall  f1-score   support

        none       1.00      0.38      0.55      8700
       dcopy       0.06      0.99      0.12       225
    memeater       1.00      1.00      1.00       225
        leak       1.00      1.00      1.00       220
        dial       0.06      0.61      0.11       220

    accuracy                           0.43      9590
   macro avg       0.62      0.80      0.56      9590
weighted avg       0.95      0.43      0.55      9590

2022-02-18 19:03:38,356 INFO    train:cg-miniGhost#test:sp-bt-miniMD-kripke-CoMD#model:RandomForestClassifier#cv:0
2022-02-18 19:03:38,373 INFO    Total misclassified normal runs: 5400, Total normal runs 8700 
2022-02-18 19:03:38,373 INFO    FAR: 0.6206896551724138
2022-02-18 19:03:38,374 INFO    Total misclassified anom runs: 6, Total anom runs 890 
2022-02-18 19:03:38,375 INFO    AMR: 0.006741573033707865
2022-02-18 19:03:38,377 INFO    Selected train apps: ['cg', 'lu']


2022-02-18 19:04:33,071 INFO    train:ft-cg-miniAMR-mg#test:sp-bt-miniMD-kripke-CoMD#model:RandomForestClassifier#cv:0
2022-02-18 19:04:33,087 INFO    Total misclassified normal runs: 365, Total normal runs 8700 
2022-02-18 19:04:33,088 INFO    FAR: 0.04195402298850575
2022-02-18 19:04:33,089 INFO    Total misclassified anom runs: 96, Total anom runs 890 
2022-02-18 19:04:33,090 INFO    AMR: 0.10786516853932585
2022-02-18 19:04:33,091 INFO    Selected train apps: ['ft', 'cg', 'miniAMR', 'miniGhost']
2022-02-18 19:04:46,941 INFO    
               precision    recall  f1-score   support

        none       0.99      0.96      0.97      8700
       dcopy       0.98      1.00      0.99       225
    memeater       1.00      1.00      1.00       225
        leak       0.99      1.00      1.00       220
        dial       0.27      0.59      0.37       220

    accuracy                           0.95      9590
   macro avg       0.85      0.91      0.87      9590
weighted avg       0.97    

2022-02-18 19:06:23,980 INFO    train:ft-miniAMR-miniGhost-lu#test:sp-bt-miniMD-kripke-CoMD#model:RandomForestClassifier#cv:0
2022-02-18 19:06:23,997 INFO    Total misclassified normal runs: 309, Total normal runs 8700 
2022-02-18 19:06:23,997 INFO    FAR: 0.035517241379310345
2022-02-18 19:06:23,998 INFO    Total misclassified anom runs: 85, Total anom runs 890 
2022-02-18 19:06:23,999 INFO    AMR: 0.09550561797752809
2022-02-18 19:06:24,001 INFO    Selected train apps: ['ft', 'mg', 'miniGhost', 'lu']
2022-02-18 19:06:38,624 INFO    
               precision    recall  f1-score   support

        none       0.99      0.90      0.95      8700
       dcopy       0.95      1.00      0.97       225
    memeater       1.00      1.00      1.00       225
        leak       1.00      1.00      1.00       220
        dial       0.15      0.65      0.24       220

    accuracy                           0.91      9590
   macro avg       0.82      0.91      0.83      9590
weighted avg       0.97 

### 5-fold CV Validation

In [29]:
SCALER = 'MinMax'

if SCALER == 'MinMax':
    
    scaler = MinMaxScaler()    

elif SCALER == 'Standard':
    
    scaler = StandardScaler()

clf = RandomForestClassifier(random_state=0)

pipeline = Pipeline([('transformer', scaler), ('estimator', clf)])

In [31]:
scoring = ['precision_macro', 'recall_macro', 'f1_macro', 'f1_weighted']
skf = StratifiedKFold(n_splits=5)

scores = cross_validate(pipeline, all_data, all_label['anom'].values,                         
                        cv=skf, 
                        scoring=scoring)

In [32]:
scores

{'fit_time': array([37.87093496, 33.30407429, 31.670362  , 32.61878467, 33.78410292]),
 'score_time': array([0.22262263, 0.16374779, 0.16079235, 0.16500545, 0.16272569]),
 'test_precision_macro': array([0.99701681, 0.9971924 , 0.99916055, 0.99936975, 0.99889966]),
 'test_recall_macro': array([0.97295918, 0.96869731, 0.96666667, 0.975     , 0.95625   ]),
 'test_f1_macro': array([0.98429409, 0.9816149 , 0.98139757, 0.98635104, 0.97488691]),
 'test_f1_weighted': array([0.9967994 , 0.99602717, 0.99600515, 0.99703788, 0.99467834])}